In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals  

%matplotlib inline
%load_ext autoreload
%autoreload 2

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model

import scipy
import sklearn

sns.set(color_codes=True)

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base

import os
import sys
sys.path.append("/home/verena/deployment/interpretable-audio-models/iml_methods/influence-release/")
from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
from influence.smooth_hinge import SmoothHinge
import influence.dataset as dataset
from influence.dataset import DataSet
#from binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
#from smooth_hinge import SmoothHinge
#import dataset as dataset
#from dataset import DataSet

np.random.seed(42)

Using TensorFlow backend.


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
def examine_vec(x, verbose=False):
    assert len(feature_names) == len(x)
    print('Age: %s' % x[age_var_indices])
    if verbose:
        for feature_name, val in zip(feature_names, x):
            print('%32s: %.6f' % (feature_name, val))
    
def examine_train_point(idx, verbose=False):
    print('Label: %s' % Y_train[idx])
    examine_vec(modified_X_train[idx, :], verbose)
    
def examine_test_point(idx, verbose=False):
    print('Label: %s' % Y_test[idx])
    examine_vec(X_test[idx, :], verbose)

# Read and process dataset

In [ ]:
df = pd.read_csv('/home/verena/experiments/influence/diabetic_data.csv') # TODO: dynamic path
# Use this if you are not running this in CodaLab
# df = pd.read_csv('../data/diabetic_data.csv')

In [ ]:
df.head()

In [ ]:
# Convert categorical variables into numeric ones

X = pd.DataFrame()

# Numerical variables that we can pull directly
X = df.loc[:,  
    [
        'time_in_hospital',
        'num_lab_procedures',
        'num_procedures',
        'num_medications',
        'number_outpatient',
        'number_emergency',
        'number_inpatient',
        'number_diagnoses'
    ]]

categorical_var_names = [
    'gender',
    'race',
    'age', 
    'discharge_disposition_id',
    'max_glu_serum',
    'A1Cresult',
    'metformin',
    'repaglinide',
    'nateglinide',
    'chlorpropamide',
    'glimepiride',
    'acetohexamide',
    'glipizide',
    'glyburide',
    'tolbutamide',
    'pioglitazone',
    'rosiglitazone',
    'acarbose',
    'miglitol',
    'troglitazone',
    'tolazamide',
    'examide',
    'citoglipton',
    'insulin',
    'glyburide-metformin',
    'glipizide-metformin',
    'glimepiride-pioglitazone',
    'metformin-rosiglitazone',
    'metformin-pioglitazone',
    'change',
    'diabetesMed'
]

for categorical_var_name in categorical_var_names:
    categorical_var = pd.Categorical(
        df.loc[:, categorical_var_name])
    
    # Just have one dummy variable if it's boolean
    if len(categorical_var.categories) == 2:
        drop_first = True
    else:
        drop_first = False

    dummies = pd.get_dummies(
        categorical_var, 
        prefix=categorical_var_name,
        drop_first=drop_first)
    
    X = pd.concat([X, dummies], axis=1)

In [ ]:
### Set the Y labels
readmitted = pd.Categorical(df.readmitted)

Y = np.copy(readmitted.codes)

# Combine >30 and 0 and flip labels, so 1 (>30) and 2 (No) become -1, while 0 becomes 1
Y[Y >= 1] = -1
Y[Y == 0] = 1

# Map to feature names
feature_names = X.columns.values

### Find indices of age features
age_var = pd.Categorical(df.loc[:, 'age'])
age_var_names = ['age_%s' % age_var_name for age_var_name in age_var.categories]    
age_var_indices = []
for age_var_name in age_var_names:
    age_var_indices.append(np.where(X.columns.values == age_var_name)[0][0])
age_var_indices = np.array(age_var_indices, dtype=int)

In [ ]:
### Split into training and test sets. 
# For convenience, we balance the training set to have 10k positives and 10k negatives.

np.random.seed(2)
num_examples = len(Y)
assert X.shape[0] == num_examples
num_train_examples = 20000
num_train_examples_per_class = int(num_train_examples / 2)
num_test_examples = num_examples - num_train_examples
assert num_test_examples > 0

pos_idx = np.where(Y == 1)[0]
neg_idx = np.where(Y == -1)[0]
np.random.shuffle(pos_idx)
np.random.shuffle(neg_idx)
assert len(pos_idx) + len(neg_idx) == num_examples

train_idx = np.concatenate((pos_idx[:num_train_examples_per_class], neg_idx[:num_train_examples_per_class]))
test_idx = np.concatenate((pos_idx[num_train_examples_per_class:], neg_idx[num_train_examples_per_class:]))
np.random.shuffle(train_idx)
np.random.shuffle(test_idx)

X_train = np.array(X.iloc[train_idx, :], dtype=np.float32)
Y_train = Y[train_idx]

X_test = np.array(X.iloc[test_idx, :], dtype=np.float32)
Y_test = Y[test_idx]

#train = DataSet(X_train, Y_train)
#validation = None
#test = DataSet(X_test, Y_test)
#data_sets = base.Datasets(train=train, validation=validation, test=test)

lr_train = DataSet(X_train, np.array((Y_train + 1) / 2, dtype=int))
lr_validation = None
lr_test = DataSet(X_test, np.array((Y_test + 1) / 2, dtype=int))
lr_data_sets = base.Datasets(train=lr_train, validation=lr_validation, test=lr_test)

test_children_idx = np.where(X_test[:, age_var_indices[0]] == 1)[0]

In [ ]:
np.unique((Y_test + 1) / 2)

In [ ]:
# Train a model on the training set

num_classes = 2

input_dim = X_train.shape[1]
weight_decay = 0.0001
batch_size = 100
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]
max_lbfgs_iter = 1000
use_bias = True

tf.reset_default_graph()

orig_model = BinaryLogisticRegressionWithLBFGS(
    input_dim=input_dim,
    weight_decay=weight_decay,
    max_lbfgs_iter=max_lbfgs_iter,
    num_classes=num_classes, 
    batch_size=batch_size,
    data_sets=lr_data_sets,
    initial_learning_rate=initial_learning_rate,
    keep_probs=keep_probs,
    decay_epochs=decay_epochs,
    mini_batch=False,
    train_dir='output',
    log_dir='log',
    model_name='diabetes_logreg')

orig_model.train()

orig_model_preds = orig_model.sess.run(
    orig_model.preds,
    feed_dict=orig_model.all_test_feed_dict)
print(orig_model_preds)
orig_model_preds = orig_model_preds[test_children_idx, 0]

In [ ]:
input_dim

In [ ]:
# Remove from the training set all but one young patients who didn't get readmitted 
mask_to_remove = (Y_train == -1) & (X_train[:, age_var_indices[0]] == 1) 
idx_to_remove = np.where(mask_to_remove)[0][:-1] # Keep 1 of them
mask_to_keep = np.array([True] * len(mask_to_remove), dtype=bool)
mask_to_keep[idx_to_remove] = False

modified_X_train = np.copy(X_train)
modified_Y_train = np.copy(Y_train)

modified_X_train = modified_X_train[mask_to_keep, :]
modified_Y_train = modified_Y_train[mask_to_keep]

print('In original data, %s/%s children were readmitted.' % (
        np.sum((Y_train == 1) & (X_train[:, age_var_indices[0]] == 1)),
        np.sum((X_train[:, age_var_indices[0]] == 1))))
print('In modified data, %s/%s children were readmitted.' % (
        np.sum((modified_Y_train == 1) & (modified_X_train[:, age_var_indices[0]] == 1)),
        np.sum((modified_X_train[:, age_var_indices[0]] == 1))))

#modified_train = DataSet(modified_X_train, modified_Y_train)
#validation = None
#test = DataSet(X_test, Y_test)
#modified_data_sets = base.Datasets(train=modified_train, validation=validation, test=test)


lr_modified_train = DataSet(modified_X_train, np.array((modified_Y_train + 1) / 2, dtype=int))
lr_modified_data_sets = base.Datasets(train=lr_modified_train, validation=lr_validation, test=lr_test)

In [ ]:
# Train a model on the modified training set
tf.reset_default_graph()

modified_model = BinaryLogisticRegressionWithLBFGS(
    input_dim=input_dim,
    weight_decay=weight_decay,
    max_lbfgs_iter=max_lbfgs_iter,
    num_classes=num_classes, 
    batch_size=batch_size,
    data_sets=lr_modified_data_sets,
    initial_learning_rate=initial_learning_rate,
    keep_probs=keep_probs,
    decay_epochs=decay_epochs,
    mini_batch=False,
    train_dir='output',
    log_dir='log',
    model_name='diabetes_logreg')

modified_model.train()

modified_model_preds = modified_model.sess.run(
    modified_model.preds,
    feed_dict=modified_model.all_test_feed_dict)
modified_model_preds = modified_model_preds[test_children_idx, 0]
modified_theta = modified_model.sess.run(modified_model.params)[0]

In [ ]:
# Baseline: look at coefficient values
sns.set_style('white')
plt.figure(figsize=(8, 10))
idx = np.argsort(np.abs(modified_theta))[-20:]
sns.barplot(np.abs(modified_theta[idx]), X.columns.values[idx])

In [ ]:
# Find children in the test set and see how predictions change on them
true_labels = Y_test[test_children_idx]

for i in range(len(test_children_idx)):
    if (orig_model_preds[i] < 0.5) != (modified_model_preds[i] < 0.5):
        print('*** ', end='')
    print("index %s, label %s: %s vs. %s" % (
        test_children_idx[i], true_labels[i], 
        orig_model_preds[i], modified_model_preds[i]))

In [ ]:
# Pick one of those children and find the most influential examples on it
test_idx = 1742
x_test = X_test[test_idx, :]
y_test = Y_test[test_idx]
print("Test point features:")
print(x_test)
print(y_test)
print('Younger than 10? %s' % x_test[age_var_indices[0]])

# Dissect get_influence_on_test_loss

In [ ]:
import time

In [ ]:
sys.path.append("../influence/")
from ihvp import get_inverse_hvp_cg

In [ ]:
def get_influence_on_test_loss(trained_model, test_indices, train_idx, 
    approx_type='cg', approx_params=None, force_refresh=True, test_description=None,
    loss_type='normal_loss',
    X=None, Y=None):
    # If train_idx is None then use X and Y (phantom points)
    # Need to make sure test_idx stays consistent between models
    # because mini-batching permutes dataset order

    if train_idx is None: 
        if (X is None) or (Y is None): raise ValueError('X and Y must be specified if using phantom points.')
        if X.shape[0] != len(Y): raise ValueError('X and Y must have the same length.')
    else:
        if (X is not None) or (Y is not None): raise ValueError('X and Y cannot be specified if train_idx is specified.')
    
    test_grad_loss_no_reg_val = trained_model.get_test_grad_loss_no_reg_val(test_indices, loss_type=loss_type)

    print('Norm of test gradient: %s' % np.linalg.norm(np.concatenate(test_grad_loss_no_reg_val)))

    start_time = time.time()

    if test_description is None:
        test_description = test_indices

    #approx_filename = os.path.join(trained_model.train_dir, '%s-%s-%s-test-%s.npz' % (self.model_name, approx_type, loss_type, test_description))
    #if os.path.exists(approx_filename) and force_refresh == False:
    #    inverse_hvp = list(np.load(approx_filename)['inverse_hvp'])
    #    print('Loaded inverse HVP from %s' % approx_filename)
    #else:
    inverse_hvp = get_inverse_hvp_cg(
        modified_model,
        test_grad_loss_no_reg_val# ,
        # approx_type,
        # approx_params
    )
    #    np.savez(approx_filename, inverse_hvp=inverse_hvp)
    #    print('Saved inverse HVP to %s' % approx_filename)
    np.save("inverse_hvp_tf.npy", inverse_hvp)
    duration = time.time() - start_time
    print('Inverse HVP took %s sec' % duration)


    print("Number of training examples", trained_model.num_train_examples)
    start_time = time.time()
    if train_idx is None:
        num_to_remove = len(Y)
        predicted_loss_diffs = np.zeros([num_to_remove])            
        for counter in np.arange(num_to_remove):
            single_train_feed_dict = trained_model.fill_feed_dict_manual(X[counter, :], [Y[counter]])      
            train_grad_loss_val = trained_model.sess.run(trained_model.grad_total_loss_op, feed_dict=single_train_feed_dict)
            predicted_loss_diffs[counter] = np.dot(np.concatenate(inverse_hvp), np.concatenate(train_grad_loss_val)) / trained_model.num_train_examples            

    else:            
        num_to_remove = len(train_idx)
        train_grad_loss_list = np.zeros([num_to_remove, 127])
        predicted_loss_diffs = np.zeros([num_to_remove])
        for counter, idx_to_remove in enumerate(train_idx):            
            single_train_feed_dict = trained_model.fill_feed_dict_with_one_ex(trained_model.data_sets.train, idx_to_remove)      
            train_grad_loss_val = trained_model.sess.run(trained_model.grad_total_loss_op, feed_dict=single_train_feed_dict)
            predicted_loss_diffs[counter] = np.dot(np.concatenate(inverse_hvp), np.concatenate(train_grad_loss_val)) / trained_model.num_train_examples
            # print(train_grad_loss_val[0])
            train_grad_loss_list[counter, :] = train_grad_loss_val[0]
    
    np.save("train_grad_tf.npy", train_grad_loss_list)
    print("train_grad_loss_list", train_grad_loss_list)
    duration = time.time() - start_time
    print('Multiplying by %s train examples took %s sec' % (num_to_remove, duration))

    return predicted_loss_diffs

In [ ]:
# modified_model.sess.run(modified_model.v_placeholder)

In [ ]:
# import cProfile

In [ ]:
influences = get_influence_on_test_loss(modified_model,
                                        test_indices=[1742]
                                        ,train_idx=np.arange(len(modified_model.data_sets.train.labels))
                                        ,force_refresh=False
                                       )

In [ ]:
influences[:10]

In [ ]:
len(modified_model.data_sets.train.labels)

# Continue with original code

In [ ]:
import numpy as np

In [ ]:
top_k = 10
helpful_points = np.argsort(influences)[-top_k:][::-1]
unhelpful_points = np.argsort(influences)[:top_k]

influences_to_plot = []
ages_to_plot = []

for points, message in [
    (unhelpful_points, 'worse'), (helpful_points, 'better')]:
    print("Top %s training points making the loss on the test point %s:" % (top_k, message))
    for counter, idx in enumerate(points):
        print("#%5d, class=%s, age=%s, predicted_loss_diff=%.8f" % (
            idx,                 
            modified_Y_train[idx], 
            modified_X_train[idx, age_var_indices],
            influences[idx]))
        
        ages_to_plot.append(idx)
        influences_to_plot.append(influences[idx])


In [ ]:
# The children in the modified dataset are by far the most influential
plt.figure(figsize=(15,6))
sort_idx = np.argsort(influences_to_plot)
ages_to_plot = np.array(ages_to_plot)
sns.barplot(ages_to_plot, influences_to_plot, order=ages_to_plot[sort_idx])

In [ ]:
def get_grad_of_influence_wrt_input(trained_model, train_indices, test_indices, 
    approx_type='cg', approx_params=None, force_refresh=True, verbose=True, test_description=None,
    loss_type='normal_loss'):
    """
    If the loss goes up when you remove a point, then it was a helpful point.
    So positive influence = helpful.
    If we move in the direction of the gradient, we make the influence even more positive, 
    so even more helpful.
    Thus if we want to make the test point more wrong, we have to move in the opposite direction.
    """

    # Calculate v_placeholder (gradient of loss at test point)
    test_grad_loss_no_reg_val = trained_model.get_test_grad_loss_no_reg_val(test_indices, loss_type=loss_type)            

    if verbose: print('Norm of test gradient: %s' % np.linalg.norm(np.concatenate(test_grad_loss_no_reg_val)))

    start_time = time.time()

    if test_description is None:
        test_description = test_indices

    #approx_filename = os.path.join(trained_model.train_dir, '%s-%s-%s-test-%s.npz' % (trained_model.model_name, approx_type, loss_type, test_description))

    #if os.path.exists(approx_filename) and force_refresh == False:
    #    inverse_hvp = list(np.load(approx_filename)['inverse_hvp'])
    #    if verbose: print('Loaded inverse HVP from %s' % approx_filename)
    #else:            
    inverse_hvp = trained_model.get_inverse_hvp(
        test_grad_loss_no_reg_val,
        approx_type,
        approx_params,
        verbose=verbose)
    #np.savez(approx_filename, inverse_hvp=inverse_hvp)
    #if verbose: print('Saved inverse HVP to %s' % approx_filename)            

    duration = time.time() - start_time
    if verbose: print('Inverse HVP took %s sec' % duration)

    grad_influence_wrt_input_val = None

    for counter, train_idx in enumerate(train_indices):
        # Put in the train example in the feed dict
        grad_influence_feed_dict = trained_model.fill_feed_dict_with_one_ex(
            trained_model.data_sets.train,  
            train_idx)

        trained_model.update_feed_dict_with_v_placeholder(grad_influence_feed_dict, inverse_hvp)
        
        #print("grad_influence_feed_dict", trained_model.sess.run(grad_influence_feed_dict))
        
        # Run the grad op with the feed dict
        current_grad_influence_wrt_input_val = trained_model.sess.run(trained_model.grad_influence_wrt_input_op, feed_dict=grad_influence_feed_dict)[0][0, :]            

        if grad_influence_wrt_input_val is None:
            grad_influence_wrt_input_val = np.zeros([len(train_indices), len(current_grad_influence_wrt_input_val)])

        grad_influence_wrt_input_val[counter, :] = current_grad_influence_wrt_input_val

    return grad_influence_wrt_input_val

In [ ]:
# Look at which features are causing this influence
grad_influences_wrt_input_val = get_grad_of_influence_wrt_input(
    modified_model,
    [19590, 13685, 9366, 11116], 
    [test_idx], 
    force_refresh=False,
    test_description=None,
    loss_type='normal_loss')    

delta = grad_influences_wrt_input_val[0, :]
plt.figure(figsize=(8, 6))
idx_to_plot = np.array([0] * len(delta), dtype=bool)
idx_to_plot[:10] = 1
idx_to_plot[-10:] = 1
sns.barplot(np.sort(delta)[idx_to_plot], feature_names[np.argsort(delta)[idx_to_plot]], orient='h')

In [ ]:
approx_filename = 'output/diabetes_logreg-cg-normal_loss-test-[1742].npz'
inverse_hvp = list(np.load(approx_filename)['inverse_hvp'])
inverse_hvp

In [ ]:
grad_influences_wrt_input_val.shape